In [2]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
import keras
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, RepeatVector
from keras.optimizers import RMSprop, SGD
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
proteins = ["*"]
structures = ["*"]



path = "output5597.out"
with open(path) as f:
    for line in f:
        splited = line.strip().split(' ')
        if len(list(splited[1])) == len(list(splited[2])):
            proteins.extend(list(splited[1].upper()))
            proteins.extend(['*']*10)
            structures.extend(list(splited[2].upper()))
            structures.extend(['*']*10)
        
            
            
alphabet_structures = ['C', 'H', 'E', 'T', "*"]
alphabet_proteins = ['A','R','N','D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', '*']

known_proteins = []
known_structures = []
for i in range(len(proteins)):
    if proteins[i] in alphabet_proteins and structures[i] in alphabet_structures:
        known_proteins.append(proteins[i])
        known_structures.append(structures[i])
proteins = known_proteins
structures = known_structures

structures_indices = dict((c, i) for i, c in enumerate(alphabet_structures))
indices_structures = dict((i, c) for i, c in enumerate(alphabet_structures))

proteins_indices = dict((c, i) for i, c in enumerate(alphabet_proteins))
indices_proteins = dict((i, c) for i, c in enumerate(alphabet_proteins))


In [16]:

for mem_depth in [12,16,20]:

    #Get time series
    protein_blocks = []
    structure = []
    for i in range(0, len(proteins) - mem_depth + 1):
        protein_blocks.append(proteins[i: i + mem_depth])
        structure.append(structures[i + int(mem_depth/2) - 1])


    #Vectorisation
    X = np.zeros((len(protein_blocks), mem_depth, len(alphabet_proteins)), dtype=np.bool)
    y = np.zeros((len(structure), len(alphabet_structures)), dtype=np.bool)
    for i, block in enumerate(protein_blocks):
        for t, protein in enumerate(block):
            X[i, t, proteins_indices[protein]] = 1
        y[i, structures_indices[structure[i]]] = 1

    
    for nodes in [64,128,256]:
        for learning_rate in [0.1, 0.01, 0.001]:
            for momentum_val in [0.3, 0.6, 0.9]:
                print('Build model...')
                
                
                print("Nodes: {0} LR: {1} MV: {2}".format(nodes, learning_rate, momentum_val))
                for i in range(3):
                    
                    model = Sequential()
                    model.add(Bidirectional(GRU(nodes), input_shape=(mem_depth, len(alphabet_proteins))))
                    model.add(Dense(len(alphabet_structures)))
                    model.add(Activation('softmax'))

                    optimizer = SGD(lr=learning_rate, momentum=momentum_val)

                    model.compile(loss='categorical_crossentropy', optimizer=optimizer) 
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.1)
                    model.fit(X_train, y_train, batch_size=128, epochs=5)
                    test_filtered_x = np.array([X_test[i] for i in range(len(X_test)) if alphabet_structures[np.argmax(y_test[i])] != '*'])
                    test_filtered_y = np.array([y_test[i] for i in range(len(y_test)) if alphabet_structures[np.argmax(y_test[i])] != '*'])
                    predictions = (model.predict(test_filtered_x))
                    predictions = [alphabet_structures[np.argmax(prediction)] for prediction in predictions]
                    test_filtered_y = [alphabet_structures[np.argmax(y)] for y in test_filtered_y]
                    print("TEST SET ACC:")
                    print(np.mean(np.array(predictions) == test_filtered_y))


Wynik w pliku Eval_stage_1_result.txt

Obiecujące wyniki to wszystkie o LR = 0.1 oraz te spośród LR=0.01, które mają momentum = 0.9

In [17]:
##Etap drugi, sprawdzenie przypadków LR = 0.01, momentum = 0.9 na podstawie 10 epok.

for mem_depth in [12,16,20]:

    #Get time series
    protein_blocks = []
    structure = []
    for i in range(0, len(proteins) - mem_depth + 1):
        protein_blocks.append(proteins[i: i + mem_depth])
        structure.append(structures[i + int(mem_depth/2) - 1])


    #Vectorisation
    X = np.zeros((len(protein_blocks), mem_depth, len(alphabet_proteins)), dtype=np.bool)
    y = np.zeros((len(structure), len(alphabet_structures)), dtype=np.bool)
    for i, block in enumerate(protein_blocks):
        for t, protein in enumerate(block):
            X[i, t, proteins_indices[protein]] = 1
        y[i, structures_indices[structure[i]]] = 1

    
    for nodes in [64,128,256]:
        for learning_rate in [0.01]:
            for momentum_val in [0.9]:
                print('Build model...')
                
                
                print("Nodes: {0} LR: {1} MV: {2}".format(nodes, learning_rate, momentum_val))
                for i in range(3):
                    
                    model = Sequential()
                    model.add(Bidirectional(GRU(nodes), input_shape=(mem_depth, len(alphabet_proteins))))
                    model.add(Dense(len(alphabet_structures)))
                    model.add(Activation('softmax'))

                    optimizer = SGD(lr=learning_rate, momentum=momentum_val)

                    model.compile(loss='categorical_crossentropy', optimizer=optimizer) 
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.1)
                    model.fit(X_train, y_train, batch_size=128, epochs=10)
                    test_filtered_x = np.array([X_test[i] for i in range(len(X_test)) if alphabet_structures[np.argmax(y_test[i])] != '*'])
                    test_filtered_y = np.array([y_test[i] for i in range(len(y_test)) if alphabet_structures[np.argmax(y_test[i])] != '*'])
                    predictions = (model.predict(test_filtered_x))
                    predictions = [alphabet_structures[np.argmax(prediction)] for prediction in predictions]
                    test_filtered_y = [alphabet_structures[np.argmax(y)] for y in test_filtered_y]
                    print("TEST SET ACC:")
                    print(np.mean(np.array(predictions) == test_filtered_y))

Build model...
Nodes: 64 LR: 0.01 MV: 0.9
Epoch 1/10
113198/113198 [==============================] - 24s - loss: 1.2617    
Epoch 2/10
113198/113198 [==============================] - 25s - loss: 1.1369    
Epoch 3/10
113198/113198 [==============================] - 25s - loss: 1.1087    
Epoch 4/10
113198/113198 [==============================] - 24s - loss: 1.0912    
Epoch 5/10
113198/113198 [==============================] - 25s - loss: 1.0794    
Epoch 6/10
113198/113198 [==============================] - 24s - loss: 1.0718    
Epoch 7/10
113198/113198 [==============================] - 25s - loss: 1.0662    
Epoch 8/10
113198/113198 [==============================] - 24s - loss: 1.0614    
Epoch 9/10
113198/113198 [==============================] - 25s - loss: 1.0577    
Epoch 10/10
113198/113198 [==============================] - 24s - loss: 1.0550    
TEST SET ACC:
0.525063145383
Epoch 1/10
113198/113198 [==============================] - 23s - loss: 1.2651    
Epoch 2/10
1131

KeyboardInterrupt: 